In [126]:
#установка не самых очевидных библиотек
pip install pycountry-convert
pip install flatten_json

  Created wheel for pycountry: filename=pycountry-20.7.3-py2.py3-none-any.whl size=10746871 sha256=3aa17fb357f407c6ac3c897e6fab76966d5833f15f7ee80c616f4bb9b7593f2e
  Stored in directory: C:\Users\vasily\AppData\Local\pip\Cache\wheels\33\4e\a6\be297e6b83567e537bed9df4a93f8590ec01c1acfbcd405348
Successfully built pycountry
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import numpy as np
import os
import json
from pandas.io.json import json_normalize
##flattening json with credits to https://towardsdatascience.com/flattening-json-objects-in-python-f5343c794b10
from flatten_json import flatten
import pycountry_convert 
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
#открываю файл локально(
with open('nlp_dataset_2000_2019.json', encoding='utf-8') as f:
    j_data = json.load(f)

In [56]:
#оформляем df для табло. 
not_fully_flat_df = json_normalize(j_data)

In [692]:
list(not_fully_flat_df)

['_index',
 '_type',
 '_id',
 '_score',
 '_source.DOI',
 '_source.issue',
 '_source.volume',
 '_source.created',
 '_source.pubdate',
 '_source.doc_type',
 '_source.page_end',
 '_source.publisher',
 '_source.book_title',
 '_source.page_start',
 '_source.citation_count',
 '_source.reference_count',
 '_source.nid',
 '_source.fields_of_study',
 '_source.fos',
 '_source.language',
 '_source.title.en',
 '_source.has_title',
 '_source.organisations',
 '_source.affiliations',
 '_source.authors',
 '_source.has_abstract',
 '_source.abstract.en',
 '_source.phrases.en.part.title.ORG',
 '_source.phrases.en.part.abstract.NP',
 '_source.phrases.en.part.abstract.ORG',
 '_source.phrases.en.model.name',
 '_source.phrases.en.model.version',
 '_source.embedding.en.bert.model',
 '_source.embedding.en.bert.vector',
 '_source.countries',
 '_source.phrases.en.part.title.NP',
 '_source.phrases.en.part.abstract.PRODUCT',
 '_source.phrases.en.part.title.GPE',
 '_source.title.es',
 '_source.phrases.en.part.abstra

In [57]:
#попробуем получить информацию и недостающих странах из информации об унверситетах
#функция, проходит по списку словарей, вытаскивает из каждого код страны  заполняет ими список 
def country_getter(l):
    c_list = []
    for dic in l:
        if 'country_code' in list(dic):
            c_list.append(dic['country_code'])
    return list(set(c_list))
affl_countries = not_fully_flat_df['_source.affiliations'].dropna().apply(country_getter)    
new_rows = abs(not_fully_flat_df['_source.countries'].notna().sum() - affl_countries.notna().sum())
print("Ура! Можно добавить информацию по странам для {} наблюдений ".format(new_rows))
not_fully_flat_df['_source.countries'].update(affl_countries)

Ура! Можно добавить информацию по странам для 37 наблюдений 


In [58]:
def org_getter(l):
    c_list = []
    for dic in l:
        if 'name' in list(dic):
            c_list.append(dic['name'])
    return list(set(c_list))
affl_orgs = not_fully_flat_df['_source.affiliations'].dropna().apply(org_getter)
if affl_orgs.notna().sum() == not_fully_flat_df['_source.organisations'].notna().sum(): 
    print('В данных по organisations и affiliations расхождений нет') 
#если расхождений ет, то можно и не обновлять колонку organisations.    

В данных по organisations и affiliations расхождений нет


In [59]:
#теперь займемся fos тэгами
#кол-во уникальных fos тэгов на 1 статью 
not_fully_flat_df['_source.fos'].apply(len).value_counts()
#было бы удобнее, если бы это количество было единым для всех наблюдений...

11    5504
10    4469
12    3629
9     2566
8     2265
7     1777
6     1310
13     887
5      761
4      369
3      135
14      93
2       14
15       4
1        2
Name: _source.fos, dtype: int64

In [276]:
#распаковать все термины в один Series
list_of_terms = list(not_fully_flat_df['_source.fos'])
full_terms = pd.Series([y for x in list_of_terms for y in x])
#топ 20 популярных терминов и их частота появления в статьях
full_terms.value_counts().head(20)
#то же самое одной строчкой (ОСТАВИТЬ ПОТОМ ЭТУ СТРОЧКУ)
not_fully_flat_df['_source.fos'].explode().value_counts().head(20)

Computer science               18701
Artificial intelligence        12663
Natural language processing     7974
Text mining                     6962
Data mining                     6933
Machine learning                4510
Information retrieval           3410
Pattern recognition             2290
Natural language                2040
Data science                    1582
Information extraction          1504
Sentiment analysis              1431
Cluster analysis                1285
Sentence                        1163
Artificial neural network       1161
Semantics                       1138
Parsing                         1064
Syntax                          1032
The Internet                    1005
Social media                     997
Name: _source.fos, dtype: int64

In [61]:
#При этом всего разных видов тэгов
print('В наблюдениях содержится {} типов fos-тэгов'.format(len(full_terms.value_counts())))

В наблюдениях содержится 12748 типов fos-тэгов


In [62]:
#Представляется, что в данных слишком много разных fos-тэгов (в половину от кол-ва наблюдений!), кроме того для разных
#наблюдений их разное количество
#Можно попробовать оставить 4 наиболее значимых (относительно критерия score) тэга и проверить повлияет ли это на конечное их
#распределение
#фукнция оставлет в поле fields_of_study отсортированй по значению score словарь тэгов и возвращает топ-4 тега (ключи словаря)
def fos_rank(l):
    rank = {}
    for dic in l:
        rank[dic['name']] = dic['score']
    rank = {x: y for x, y in sorted(rank.items(), key = lambda x: x[1], reverse = True)[0:4]}
    return list(rank)
not_fully_flat_df['_source.fields_of_study'].apply(fos_rank)


0        [Biomedical text mining, Text mining, Temporal...
1        [Biomedical text mining, Temporal annotation, ...
2        [Text mining, Noisy text analytics, Concept mi...
3        [Relational database, Data modeling, Relationa...
4        [Language identification, Text mining, Natural...
                               ...                        
23780    [Intelligent tutoring system, Instructional de...
23781    [Linguistics, Artificial intelligence, Art, Gr...
23782    [Radial basis function, Fuzzy clustering, Fuzz...
23783    [Lexical function, Meaning–text theory, Humani...
23784    [Linea, Art, Machine learning, Artificial inte...
Name: _source.fields_of_study, Length: 23785, dtype: object

In [63]:
#добавим значения отдельной колонкой, чтобы проверить, изменятся ли результаты распределния тегов
not_fully_flat_df['ranked_fos'] = not_fully_flat_df['_source.fields_of_study'].apply(fos_rank)
#Для каждого наблюдения в таблице сейчас только 4 самых популярных тэга!

In [64]:
#распаковать все термины в один Series
list_of_ranked_terms = list(not_fully_flat_df['ranked_fos'])
full_terms_ranked = pd.Series([y for x in list_of_ranked_terms for y in x])
#топ 20 популярных терминов и их частота появления в статьях
full_terms_ranked.value_counts().head(20)
#одной строкой (ее остаивить)
not_fully_flat_df['ranked_fos'].explode().value_counts().head(30)


Text mining                       5522
Natural language processing       1893
Data mining                       1353
Sentiment analysis                1320
Information extraction            1213
Natural language                  1117
Computer science                  1052
Cluster analysis                   932
Machine translation                747
Artificial neural network          742
Deep learning                      732
Question answering                 707
Machine learning                   654
Parsing                            644
Sentence                           628
Information retrieval              599
Automatic summarization            582
Knowledge extraction               562
Named-entity recognition           560
Ontology (information science)     555
Social media                       526
Semantics                          522
Semantic similarity                508
The Internet                       488
Annotation                         468
Data science             

In [65]:
print('В наблюдениях содержится {} типов fos-тэгов'.format(len(full_terms_ranked.value_counts())))

В наблюдениях содержится 8846 типов fos-тэгов


In [217]:
#Количество уникальных тегов изменилось несильно (на 4 тысячи тегов, для тенденций это не будет играть роли)
#Однако распределение тегов заметно изменилось, теперь в топе тэги, который в большей степени напрямую связаны с NLP и 
#обработкой текстов 
#В итоговую таблицу занесем обе колонки (отражированные и нет)
full_terms_ranked.value_counts().head(50)

Text mining                       5522
Natural language processing       1893
Data mining                       1353
Sentiment analysis                1320
Information extraction            1213
Natural language                  1117
Computer science                  1052
Cluster analysis                   932
Machine translation                747
Artificial neural network          742
Deep learning                      732
Question answering                 707
Machine learning                   654
Parsing                            644
Sentence                           628
Information retrieval              599
Automatic summarization            582
Knowledge extraction               562
Named-entity recognition           560
Ontology (information science)     555
dtype: int64

In [67]:
#Оставим те тэги, из 50 наиболее популярных, которые напрямую могут быть связаны с обработкой текстовой информации
NLP_related_tags = ['Text mining', 'Natural language processing', 'Sentiment analysis', 'Information extraction', 
                        'Natural language', 'Machine translation', 'Question answering', 'Parsing', 'Sentence', 
                        'Information retrieval', 'Knowledge extraction', 'Named-entity recognition', 'Social media', 
                        'Semantics', 'Semantic similarity', 'Biomedical text mining', 'Semantic computing', 'Syntax', 
                        'Computational linguistics', 'WordNet','Language identification', 'Social network', 'Language model', 
                        'Pattern recognition', 'Web mining','Word embedding']

In [68]:
#проверим в каких работах содержатся эти теги и отмаркируем их как NLP related papers
#проверим входит ли какой либо из популярных NLP тегов в теги каждой работы (так как мы представляет каждый набор тегов 
#единой строкой, то можно будет "поймать" дополнтельные теги, состоящие из одного из слов искомого тэга, т.е. будет True,
#если среди тегов работы окажется не только Social media, но и, например, Social media mining ). Проверять будем по исходной
#колонке с тегами (неотранжированной, там инфы больше)
not_fully_flat_df['_source.fos'].apply(' '.join).str.contains('|'.join(NLP_related_tags))
paper_has_tag = not_fully_flat_df['_source.fos'].apply(' '.join).str.contains('|'.join(NLP_related_tags)).sum()
print('{} работ содержат Топ NLP-related теги'.format(paper_has_tag))

19275 работ содержат Топ NLP-related теги


In [69]:
#Проверим, не слишком ли вольно мы добавляли теги? Теперь True будет только в случае, только если среди тегов работы
#имеется любой один тег из списка Fos_NLP_tags (но условный Social media mining уже не пройдет)
#Проссумируем количество индексов, для которых истинно, что среди списка тегов этой работы, есть один из тегов спписка
#NLP_related_tags
tag_diff = len(set(not_fully_flat_df['_source.fos'].explode().isin(NLP_related_tags)[not_fully_flat_df['_source.fos']
                                         .explode().isin(NLP_related_tags)].index))
if paper_has_tag - tag_diff < 150:
    print('превышение на {} работы. Ничего страшного'.format(paper_has_tag - tag_diff))

превышение на 52 работы. Ничего страшного


In [70]:
#Добавим новую колонку в df, 1 если для работы указан один из тегов, 0 - если нет (будем использовать первое из условий)
not_fully_flat_df['nlp_related_paper'] = not_fully_flat_df['_source.fos'] \
                                       .apply(' '.join).str.contains('|'.join(NLP_related_tags)).astype(int)

In [205]:
#Теперь добавим аналогичную колонку только для тех работ, среди тегов которых есть Natural language (processing) и 
#Text mining
NLP_tags = ['Natural language','Text mining']
not_fully_flat_df['_source.fos'].apply(' '.join).str.contains('|'.join(NLP_tags)).sum()
#Использовав тег Natural language вместо Natural language processing мы дополнительно поймали порядка 700 работ, ура. 
#Код не прописываю, можно проверить добавив слово processing в словарь NLP_tags
not_fully_flat_df['nlp_paper'] = not_fully_flat_df['_source.fos'] \
                                 .apply(' '.join).str.contains('|'.join(NLP_tags)).astype(int)

In [212]:
#Можно сделать еще более жесткий критерий, и искать теги NLP и Text mining только среди отсортированных fos тегов 
not_fully_flat_df['ranked_fos'].apply(' '.join).str.contains('|'.join(NLP_tags)).sum()
not_fully_flat_df['total_nlp_paper'] = not_fully_flat_df['ranked_fos'] \
                                       .apply(' '.join).str.contains('|'.join(NLP_tags)).astype(int)

In [173]:
#Добавим также отранжированные по встречаемости (count) именные группы, извлеченные из эбстрактов работ. Извлеченные группы из 
#названий добавлять не будем, так как они доступны только для 21500 работ 
#(not_fully_flat_df['_source.phrases.en.part.title.NP'].notna().sum())
#а информация из эбстракта есть почти для всех (23511, not_fully_flat_df['_source.phrases.en.part.abstract.NP'].notna().sum())
def abs_rank(l):
    rank = {}
    for dic in l:
        rank[dic['term']] = dic['count']
    rank = {x: y for x, y in sorted(rank.items(), key = lambda x: x[1], reverse = True)[0:4]}
    return list(rank)
not_fully_flat_df['abstract4'] = not_fully_flat_df['_source.phrases.en.part.abstract.NP'] \
                                          .map(abs_rank, na_action = 'ignore')
#добавили колонку для 4 наиболее встречающихся фраз из эбстракта (или для 4 случайных фраз, если score одинаковый)


In [73]:
#Хотя здесь мы потеряем много информации (но анализировать и искать тренды будет проще)
sr = pd.Series([int(i) for i in list(not_fully_flat_df['_source.phrases.en.part.abstract.NP']
                                .dropna().apply(len).value_counts().index)]).mean()
print('В среднем для каждой работы {:.2f} ключевых фраз из эбстракта'.format(sr))
#добавим колонку с 20 ключевыми фразами, на всякий случай
def abs_rank(l):
    rank = {}
    for dic in l:
        rank[dic['term']] = dic['count']
    rank = {x: y for x, y in sorted(rank.items(), key = lambda x: x[1], reverse = True)[0:20]}
    return list(rank)
not_fully_flat_df['abstract20'] = not_fully_flat_df['_source.phrases.en.part.abstract.NP'] \
                                          .map(abs_rank, na_action = 'ignore')

В среднем для каждой работы 93.03 ключевых фраз из эбстракта


In [181]:
#сделаем сразу для самой популярной фразы в эбстракте, возможно это самая полезная информация здесь.
def abs_rank(l):
    rank = {}
    for dic in l:
        rank[dic['term']] = dic['count']
    rank = {x: y for x, y in sorted(rank.items(), key = lambda x: x[1], reverse = True)[0:1]}
    return list(rank)
not_fully_flat_df['abstract'] = not_fully_flat_df['_source.phrases.en.part.abstract.NP'] \
                                          .map(abs_rank, na_action = 'ignore')

In [74]:
#теперь попробуем скорректировать информацию по странам/ученым/организациям
#сколько в среднем стран, организаций и ученых принимали участие в написании одной работы  
print('Работы в среднем писалась в {:.2f} странах'.
      format(not_fully_flat_df['_source.countries'].map(len, na_action = 'ignore').mean()))
print('Работы в среднем писалась {:.2f} ученым'.
      format(not_fully_flat_df['_source.authors'].map(len, na_action = 'ignore').mean()))
print('Работы в среднем писалась {:.2f} организациями'.
      format(not_fully_flat_df['_source.organisations'].map(len, na_action = 'ignore').mean()))

Работы в среднем писалась в 1.31 странах
Работы в среднем писалась 3.21 ученым
Работы в среднем писалась 1.76 организациями


In [277]:
#как много стран/ученых/организаций принимали участие в написании работ
not_fully_flat_df[['_source.countries', '_source.authors', '_source.organisations']] \
                .dropna().applymap(len).apply(pd.Series.value_counts)

,_source.countries,_source.authors,_source.organisations
1,14971.0,2357,10534.0
2,3863.0,4962,5935.0
3,720.0,5053,2252.0
4,96.0,3443,732.0
5,NaN,1878,197.0
6,NaN,934,NaN
7,NaN,481,NaN
8,NaN,277,NaN
9,NaN,164,NaN
10,NaN,101,NaN


In [76]:
#Большинство работ выполнялись в рамках 1-4 стран коллективом из 1-10 ученых внутри 1-5 организаций.
#Данные об остальных работах (стран > 4, ученых > 10 и организаций > 5) представляется возможным убрать, это позволит упростить 
#структуру данных, на результат повлияет незначительно (отдельно можно рассмотреть выбросы, изучить, что за работы связаны
#с участием 85 ученых или широкого участия стран/ученых/организаций)
#Видимо на предыдущем шаге при добавлении стран из affiliations просто добавились нулевые значения(( удалим им тоже
#Сделаем копию исходника пржде чем удалять 
not_fully_flat_df_copy = not_fully_flat_df.copy()

In [77]:
#удалим работы с кол-вом стран > 4
drop_country_index = not_fully_flat_df['_source.countries'] \
                     [not_fully_flat_df['_source.countries'].map(len, na_action = 'ignore') > 4 ].index
drop_country_index = drop_country_index.append(not_fully_flat_df['_source.countries'] \
                     [not_fully_flat_df['_source.countries'].map(len, na_action = 'ignore') == 0 ].index)
print('{} наблюдения удалены'.format(len(drop_country_index)))
not_fully_flat_df.drop(drop_country_index, inplace = True)
#пересчитаем индексы
not_fully_flat_df.reset_index(drop=True, inplace = True)

110 наблюдения удалены


In [78]:
#удалим работы с кол-вом ученых > 10
drop_author_index = not_fully_flat_df['_source.authors'] \
                    [not_fully_flat_df['_source.authors'].map(len, na_action = 'ignore') > 10].index
print('{} наблюдений удалено'.format(len(drop_author_index)))
not_fully_flat_df.drop(drop_author_index, inplace = True)
not_fully_flat_df.reset_index(drop=True, inplace = True)

231 наблюдений удалено


In [79]:
#удалим работы с кол-вом организаций > 5
drop_organisation_index = not_fully_flat_df['_source.organisations'] \
                          [not_fully_flat_df['_source.organisations'].map(len, na_action = 'ignore') > 5].index
print('{} наблюдений удалено'.format(len(drop_organisation_index)))
not_fully_flat_df.drop(drop_organisation_index, inplace = True)
not_fully_flat_df.reset_index(drop=True, inplace = True)

69 наблюдений удалено


In [80]:
#сверим длину таблиц (начальной/конечной)
sum_deleted = len(drop_author_index) + len(drop_country_index) + len(drop_organisation_index)
if len(not_fully_flat_df) + sum_deleted == len(not_fully_flat_df_copy):
    print("Верно, было удалено {} наблюдений".format(sum_deleted))

Верно, было удалено 410 наблюдения


In [81]:
#проверим по таблице
not_fully_flat_df[['_source.countries', '_source.authors', '_source.organisations']] \
                .dropna().applymap(len).apply(pd.Series.value_counts)

,_source.countries,_source.authors,_source.organisations
1,14971.0,2357,10534.0
2,3863.0,4962,5935.0
3,720.0,5053,2252.0
4,96.0,3443,732.0
5,NaN,1878,197.0
6,NaN,934,NaN
7,NaN,481,NaN
8,NaN,277,NaN
9,NaN,164,NaN
10,NaN,101,NaN


In [213]:
#данные для табло (пока что без abstract20 и без abstract4, табло будет ругаться )
tableau_data = not_fully_flat_df[['_id', '_source.pubdate', '_source.doc_type','_source.publisher',
                                  '_source.citation_count', '_source.reference_count', '_source.organisations', 
                                  '_source.authors', '_source.countries', 'ranked_fos','nlp_related_paper', 'nlp_paper',
                                  'abstract', '_source.fos','total_nlp_paper']]\
               .rename(columns = {'_id':'id', '_source.pubdate':'pubdate', 
                                  '_source.doc_type':'doc_type', '_source.publisher':'publisher',
                                  '_source.citation_count':'citations', '_source.reference_count':'references', 
                                  '_source.organisations':'organisations', '_source.authors':'authors', 
                                  '_source.countries':'countries', '_source.fos':'fos'})


In [214]:
#создаем локальный файл для загрузки в табло (нужно будет заменить на свой путь; это можно сделать красивее, конечно)))
tableau_data.to_json('C:/Users/vasily/Studies/nlp_dataset_2000_2019_1.json', orient = 'records')

In [219]:
#это я думал как бы лучше загрузить файл в табло и полностью разворичивал словари из словарей в один большой df
#Из интересного ниже информация о публикациях на разных языках
#flattened nested data
flat_data = [flatten(i) for i in j_data]

In [221]:
#put into df, получаем полностью разверутый дф
flat_df = pd.DataFrame(flat_data)

In [280]:
#перевести даты в падас формат
flat_df['year'] = pd.DatetimeIndex(flat_df['_source_pubdate']).year

In [281]:
#колво публикаций в году
flat_df.groupby('year', as_index = False).aggregate({'_index': 'count'}).rename(columns = {'_index':'pub_num'})

,year,pub_num
0,2000,241
1,2001,216
2,2002,318
3,2003,332
4,2004,477
5,2005,585
6,2006,673
7,2007,789
8,2008,901
9,2009,985


In [11]:
#поиск публикаций по странам в развернутом df
countries0 = flat_df\
    .groupby('_source_countries_0', as_index = False)\
    .agg({'_index':'count'}) \
    .rename(columns = {'_index':'pub_in_country'})\
    .sort_values(by = 'pub_in_country', ascending = False)
countries0

,_source_countries_0,pub_in_country
113,US,5459
20,CN,2389
48,IN,1149
36,GB,1136
26,DE,838
...,...,...
114,UY,1
77,MW,1
69,MD,1
70,ME,1


In [282]:
#про языки.
#количество публикаций на разных языках
lang_col_list = (list(flat_df.filter(like = '_source_title_', axis = 1)))
#колво публикаций а разных языках
lang_dict = {}
for el in lang_col_list:
    lang_dict[el[-2:]] = len(flat_df) - flat_df[el].isnull().sum()
#отсортированный список с языками
sort_lang_dict = sorted(lang_dict.items(), key = lambda x : x[1])
for el in sort_lang_dict:
    if el[1] > 10:
        print('There are {} publications in {}'.format(el[1], el[0]))

There are 12 publications in de
There are 15 publications in nt
There are 20 publications in ru
There are 24 publications in el
There are 41 publications in id
There are 63 publications in pt
There are 67 publications in fr
There are 72 publications in es
There are 76 publications in ko
There are 166 publications in un
There are 23138 publications in en


In [104]:
#Это аблюдений интересно тем, что в написании этой работы приняли участие 85 человек, видимо это крупная конференция)
full_col_list = list(flat_df)
flat_df[flat_df['_source_authors_84'].notna()]

,_index,_type,_id,_score,_source_DOI,_source_issue,_source_volume,_source_created,_source_pubdate,_source_doc_type,...,_source_phrases_en_part_abstract_PERSON_127_term,_source_phrases_en_part_abstract_PERSON_128_count,_source_phrases_en_part_abstract_PERSON_128_term,_source_phrases_en_part_abstract_PERSON_129_count,_source_phrases_en_part_abstract_PERSON_129_term,_source_phrases_en_part_abstract_PERSON_130_count,_source_phrases_en_part_abstract_PERSON_130_term,_source_phrases_en_part_abstract_PERSON_131_count,_source_phrases_en_part_abstract_PERSON_131_term,year
10456,sti_science_mag_v1.0.2,_doc,2157414955,12.106363,10.1186/2041-1480-5-5,1,5,2016-06-24,2014-01-01,journal-article,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014


In [225]:
#считаем сколько раз встречается то или иное значение в таблице (про имена ученых; да про все что угодно)
flat_df[list(flat_df)].apply(lambda s: (s == 'Iryna Gurevych').sum(), axis=1).sum()
#аналогично, чуть быстрее 
#(flat_df[list(flat_df)] == 'Hongfang Liu').sum(1).sum()

26

In [289]:
#20 работ на русском языке единственные из всей базы (23850 работ), для которых указана собственная nlp модель!
not_fully_flat_df['_source.phrases.ru.model.name'].value_counts()

ru_ft_syntagrus_md    20
Name: _source.phrases.ru.model.name, dtype: int64